In [60]:
import numpy as np
from itertools import product

# Classes for neurons

In [54]:
class SensoryNeuron:
    
    def __init__(
        self,
        eye,
        orientation,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation,
        init_response,
        init_habituation
    ):
        self.eye = eye
        self.orientation = orientation
        self.alpha = alpha
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.tau_habituation = tau_habituation
        self.init_response = init_response
        self.init_habituation = init_habituation
        
    def one_timestep(
        self,
        supressive_drive
        
    ):
        _change_in_habituation = self._calculate_change_in_habituation(...)
        _change_in_response = self._calculate_change_in_response(supressive_drive)
        self.response += _change_in_response
        self.habituation += _change_in_habituation
        
    def update_response(
        self
    ):
        pass
    
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
    ):
        pass
    
    def change_habituation(
        self,
        dt
    ):
        pass

In [61]:
class SummationNeuron:
    
    def __init__(
        self,
        orientation,
        sigma,
        n,
        weight_habituation,
        tau_response,
        tau_habituation,
        init_response,
        init_habituation
    ):
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.tau_habituation = tau_habituation
        self.init_response = init_response
        self.init_habituation = init_habituation
        
    def one_timestep(
        response_left,
        response_right
    ):
        self.compute_excitatory_drive(response_left, response_right)
        _change_in_habituation = self._calculate_change_in_habituation(...)
        _change_in_response = self._calculate_change_in_response(...)
        self.response += _change_in_response
        self.habituation += _change_in_habituation
            
    def _calculate_change_in_response(
        self,
     ):
        return 
        
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        response_left,
        response_right
    ):
        self._excitatory_drive = ...
        
    @property
    def supressive_drive(
        self
    ):
        return self._excitatory_drive
    
    def _calculate_change_in_habituation(
        self  
    ):
        pass

In [62]:
class AttentionNeuron:
    
    def __init__(
        self,
        orientation,
        sigma,
        n,
        tau_reponse,
        init_response,
        init_habituation
    ):
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.init_response = init_response
        self.init_habituation = init_habituation
        
    def update_response(
        self
    ):
        pass
    
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        response_orientation_same,
        response_orientation_opposite
    ):
        self._excitatory_drive = ...

In [63]:
class OpponencyNeuron:
    
    def __init__(
        self,
        eye,
        orientation,
        sigma,
        n,
        tau_reponse,
        init_response,
        init_habituation
    ):
        self.eye = eye
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.init_response = init_response
        self.init_habituation = init_habituation
        
    def update_response(
        self
    ):
        pass
    
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        response_eye_left,
        response_eye_right
    ):
        self._excitatory_drive = ...

# Classes for populations

In [79]:
class SensoryPopulation:
    
    eyes = ['left', 'right']
    orientations = [1, 2]
    
    def __init__(
        self,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for eye, orientation in product(self.eyes, self.orientations):
            key = eye + '_' + orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:    
                _initial_habituation = np.random.rand()
            self.neurons[key] = SensoryNeuron(
                eye=eye,
                orientation=orientation,
                alpha=alpha,
                sigma=sigma,
                n=n,
                weight_opponency=weight_opponency,
                weight_attention=weight_attention,
                weight_habituation=weight_habituation,
                tau_response=tau_response,
                tau_habituation=tau_habituation,
                initial_response=initial_response,
                initial_habituation=initial_habituation
            )
            
    def one_timestep(
        self,
        _input,
        snap
    ):
        for neuron in neurons.values():
            neuron.compute_excitatory_drive(_input, snap)
        
        self.supressive_drive
        pass
    
    @property
    def supressive_drive(self):
        return sum([neuron.excitatory_drive for neuron in self.neurons.values()])

In [79]:
class SummationPopulation:
    
    orientations = [1, 2]
    
    def __init__(
        self,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for eye, orientation in product(self.eyes, self.orientations):
            key = eye + '_' + orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:    
                _initial_habituation = np.random.rand()
            self.neurons[key] = SensoryNeuron(
                eye=eye,
                orientation=orientation,
                alpha=alpha,
                sigma=sigma,
                n=n,
                weight_opponency=weight_opponency,
                weight_attention=weight_attention,
                weight_habituation=weight_habituation,
                tau_response=tau_response,
                tau_habituation=tau_habituation,
                initial_response=initial_response,
                initial_habituation=initial_habituation
            )
            
    def one_timestep(
        self,
        snapshot
    ):
        for orientation in self.orientations:
            responses = [...]
            self.neurons[orientation].update_response()
    
    @property
    def supressive_drive(self):
        return sum([neuron.excitatory_drive for neuron in self.neurons.values()])

In [81]:
class AttentionPopulation:
    
    orientations = [1,2]
    
    def __init__(
        self,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for orientation in self.orientations:
            key = orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:
                _initial_habituation = np.random.rand()
            self.neurons[key] = AttentionNeuron(
                orientation=orientation,
                sigma=sigma,
                n=n,
                tau_response=tau_response,
                initial_response=initial_response,
                initial_habituation=initial_habituation               
            )
            
    

In [67]:
class OpponencyPopulation:
    pass

In [68]:
class SummationPopulation:
    pass

# Utilities

In [23]:
def rectification():
    pass

# General structure

In [ ]:
sensory_layer = [SensoryNeuron('right', 1), SensoryNeuron('...')]
attention_layer = [...]

#define input
condition = 'switch_stimuli'
stimulus_input = get_input(condition)

def get_input(condition, n_trials):
    if condition == 'switch_stimuli':
        return np.array([0,1] * (n_trials // 2))
    else:
        raise 
        
#init_firing_rate
class Snapshot:
    def __init__(self, layers):
        ...
def extract_results(snapshots):
    pass

snaps = []
t_steps = 10000
t_delta = .01
for i in range(t_steps):
    current_time = t_delta * i
    c_input = stimulus_input[i]
    snap.append(Snapshot(sensory_layer, attention_Layer, opponency_layer))
    sensory_layer.update(snap)
    for neuron in sensory_layer:
        neuron.update_weights(c..)
    attention_layer.update_weights(snap[-1])
    opponency_layer.update_weights(snap[-1])
    
results = extract_results(snap)